In [31]:
%load_ext autoreload
%autoreload 2
import time

import osu.rulesets.beatmap as bm
import osu.rulesets.replay as rp
import numpy as np

test_name = '1hope'

test_map = f'assets/{test_name}_map.osu'
test_song = f'assets/{test_name}_song.mp3'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
# testing dt replay
# real_replay = rp.load(f'assets/ringyou_mrekk_replay.osr')
# real_replay_map = f'assets/ringyou_map.osu'
# real_replay_song = f'assets/ringyou_song.mp3'

# testing hr replay
# real_replay = rp.load(f'assets/1hope_whitecat_replay.osr')
# real_replay_map = f'assets/1hope_map.osu'
# real_replay_song = f'assets/1hope_song.mp3'

# testing ez replay
real_replay = rp.load(f'assets/ghosts_ezchamp_replay.osr')
real_replay_map = f'assets/ghosts_map.osu'
real_replay_song = f'assets/ghosts_song.mp3'

0
11403
3
3
17
36
51
68
83
101
118
134
151
169
183
201
217
234
250
269
284
302
317
334
351
368
384
400
417
434
451
469
484
501
517
534
551
567
584
601
618
634
650
668
685
695
700
717
734
751
767
784
800
818
833
850
868
884
900
918
934
950
968
986
1004
1004
1017
1035
1051
1067
1086
1101
1117
1135
1152
1167
1175
1184
1190
1201
1217
1235
1251
1265
1267
1283
1301
1317
1335
1350
1367
1384
1401
1417
1434
1450
1468
1468
1485
1501
1519
1534
1552
1568
1585
1600
1611
1617
1634
1650
1667
1685
1701
1717
1735
1751
1768
1785
1802
1817
1835
1851
1868
1884
1900
1918
1934
1951
1968
1976
1984
2001
2017
2035
2052
2067
2086
2101
2117
2134
2151
2167
2185
2186
2200
2218
2234
2252
2267
2285
2285
2301
2318
2334
2336
2352
2368
2371
11384
11386
11387
11387
11400
11413
11414
11416
11433
11450
11465
11477
11482
11491
11499
11516
11532
11549
11554
11566
11568
11582
11599
11616
11632
11633
11643
11649
11666
11683
11699
11699
11715
11722
11734
11749
11765
11782
11786
11790
11798
11816
11832
11851
11865
11876
11883
1

In [33]:
import importlib
import osu.preview.preview as preview

importlib.reload(preview)
# try previewing a real replay

preview.preview_replay(real_replay, real_replay_map, real_replay_song)

In [13]:
# data = dataset.input_data(test_map)
# data = np.reshape(data.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))
# data = torch.FloatTensor(data).to('cuda')

In [4]:
replay_data = np.load(f".generated/{test_name}.npy")
replay_data

array([[ 0.00259041,  0.07582808],
       [ 0.01636691,  0.15886295],
       [ 0.02021809,  0.17380756],
       ...,
       [ 0.00189539, -0.00471151],
       [ 0.00189539, -0.00471151],
       [ 0.0018954 , -0.00471151]], shape=(10240, 2), dtype=float32)

In [6]:


print(test_map)
# test_map.apply_mods(['hr', 'ht'])
preview.preview_replay_raw(replay_data, test_map, ['ez'], test_song)

assets/1hope_map.osu


In [ ]:
# alt_data = dataset.replay_to_output_data(test_map, test_replay)
# alt_data = np.concatenate(alt_data)
# 
# alt_data

In [ ]:
# actually play out the replay
import win32gui
import mouse
import time
import osu.rulesets.core as osu_core
import pyautogui

def active_win_name():
    hwnd = win32gui.GetForegroundWindow()
    return win32gui.GetWindowText(hwnd)

def osu_to_screen_pixel(x, y) -> (int, int):
    w, h = pyautogui.size()
    
    # screen/osu ratio
    # https://osu.ppy.sh/wiki/en/Client/Playfield
    osr_x = w / 640 * 0.75 # for some reason this constant just works. misleading wiki?
    osr_y = h / 480
    
    x = x * osr_x
    y = y * osr_y
    
    # osu width and height
    ow = osu_core.SCREEN_WIDTH * osr_x
    oh = osu_core.SCREEN_HEIGHT * osr_y
    
    offset_x = 0.5 * (w - ow)
    # "The playfield is slightly shifted vertically, placed 8 game pixels lower than the window's centre."
    offset_y = 0.5 * (h - oh) + (8 * osr_y)
    
    return int(x + offset_x), int(y + offset_y)

time.sleep(1.25)
win_name = active_win_name()

target_interval = 0.024  # 24ms per frame
start_time = time.perf_counter()
frame_count = 0

for x, y in replay_data:
    if win_name != active_win_name():
        break
    x = (x + 0.5) * osu_core.SCREEN_WIDTH
    y = (y + 0.5) * osu_core.SCREEN_HEIGHT
    (x, y) = osu_to_screen_pixel(x, y)
    mouse.move(x, y)
    
    frame_count += 1
    
    target_time = start_time + (frame_count * target_interval)
    current_time = time.perf_counter()
    
    # run at constant rate 
    sleep_time = target_time - current_time
    if sleep_time > 0:
        time.sleep(sleep_time)
